<a href="https://colab.research.google.com/github/ksolosina/BCIT/blob/master/lab3-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm

In [0]:
import pandas as pd
import numpy as np
import lightgbm # съест все сырым и построит регрессионную модель, которая покажет важные фичи
                # чтобы дальше делать лабу только на них
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

In [4]:
import io
from google.colab import files
store = files.upload() #загрузка файлов с компа в коллаб (внизу интерфейс)
store = pd.read_csv(io.BytesIO(store['googleplaystore.csv'])) #библиотека пандас позволяет создать удобный датафрейм (data)
store.head() #выведем вернхие пять записей датафрейма (по умолчанию 5)

Saving googleplaystore.csv to googleplaystore.csv


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [8]:
# Все колонки, которые не являются числами, делаем категориальными:
for column in store.select_dtypes(include = ['object']).columns.tolist():
    store[column] = store[column].astype('category')
lgbm_regressor = lightgbm.LGBMRegressor().fit(store.loc[:, store.columns != 'Rating'], store['Rating'])
lgbm_regressor # построили сырую и простую модель, вставив на X все кроме целевой, а на y - "Rating" (то что мя пытаемся предсказать)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [9]:
list_of_importances = list(zip(store.loc[:, store.columns != 'Rating'].columns.tolist(), 
                               lgbm_regressor.feature_importances_))
sorted(list_of_importances, key= lambda x: x[1], reverse= True) # список фич, отсортированных по "важности"

[('Installs', 244),
 ('App', 0),
 ('Category', 0),
 ('Reviews', 0),
 ('Size', 0),
 ('Type', 0),
 ('Price', 0),
 ('Content Rating', 0),
 ('Genres', 0),
 ('Last Updated', 0),
 ('Current Ver', 0),
 ('Android Ver', 0)]

In [10]:
important_features = [x[0] for x in sorted(list_of_importances, key= lambda x: x[1], reverse= True)[:1]]
important_features # оставим только важные фичи

['Installs']

In [0]:
important_features.extend(['Rating'])
store = store[important_features]

In [0]:
# заполнение пропусков
for column in store.select_dtypes(include = ['int64', 'float64']).columns.tolist():
    store[column] = store[column].fillna(store[column].mean())
for column in store.select_dtypes(include = ['category']).columns.tolist():
    store[column] = store[column].fillna(store[column].describe(include= ['category'])['top'])

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in store.select_dtypes(include = ['category']).columns.tolist():
    le.fit(store[column])
    store[column] = le.transform(store[column])
#кодирование категориальных признаков
store

,Rating,Installs_value_mean
0,4.100000,4.046117
1,3.900000,4.168262
2,4.700000,4.243218
3,4.500000,4.351211
4,4.300000,4.111611
5,4.400000,4.054739
6,3.800000,4.054739
7,4.100000,4.220828
8,4.400000,4.220828
9,4.700000,4.046117


In [0]:
store.loc[:, store.columns != 'Rating'] = store.loc[:, store.columns != 'Rating'].apply(lambda x: x/x.max(), axis=0)
# нормирование или масштабирование данных

In [17]:
store

,Rating,Installs_value_mean
0,4.100000,0.212954
1,3.900000,0.219382
2,4.700000,0.223327
3,4.500000,0.229011
4,4.300000,0.216401
5,4.400000,0.213407
6,3.800000,0.213407
7,4.100000,0.222149
8,4.400000,0.222149
9,4.700000,0.212954


4 лабораторная. 


In [0]:
#train
X_train, X_test, y_train, y_test = train_test_split(store.loc[:, store.columns != 'Rating'], 
                                                    store['Rating'], 
                                                    test_size= 0.1, #размер обучающей выборки 10%
                                                    random_state= 42)

In [20]:
# подбор лучшего параметра по кросс-валидации (проверка успешности модели)
random_search = GridSearchCV(estimator= KNeighborsRegressor(),
                             param_grid= {'n_neighbors': [5,10,20,50]}, #несколько вариантов параметра k
                             scoring= 'neg_mean_absolute_error') #средняя абсолютная ошибка по модели
random_search.fit(store.loc[:, store.columns != 'Rating'], store['Rating']) #обучаем fit

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [5, 10, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [22]:
random_search.best_params_ #лучшее с 50 соседями

{'n_neighbors': 50}

In [23]:
random_search.best_score_ #выведем получившуюся абсолютную ошибку

-0.3186198405365313